# 7-Slot Nerdle Solver Test
We prove (by brute-force) that you can always solve mini-Nerdle in at most $4$ guesses regardless of the starting expression, provided you use the optimal strategy. The worst start having repeating numbers and thus less information, e.g. `10-5=5`. The best start has all different numbers: `28/7=4`, which needs at most $3$ guesses and $2.65 \pm 0.5$ guesses.

In [1]:
%load_ext autoreload
%autoreload 2

import collections
import itertools
import numpy as np
import matplotlib.pyplot as plt

import nerdle
import score as s
import generator
from nerdle import Hint, NerdleData

In [4]:
# Mini-Nerdle.
NUM_SLOTS = 7
SCORE_DICT_FILE = "nerdle{}".format(NUM_SLOTS) 

solver_data = nerdle.create_solver_data(NUM_SLOTS, SCORE_DICT_FILE)

0 / 6661 (0.0%) completed
333 / 6661 (5.0%) completed
666 / 6661 (10.0%) completed
999 / 6661 (15.0%) completed
1332 / 6661 (20.0%) completed
1665 / 6661 (25.0%) completed
1998 / 6661 (30.0%) completed
2331 / 6661 (35.0%) completed
2664 / 6661 (40.0%) completed
2997 / 6661 (45.0%) completed
3330 / 6661 (50.0%) completed
3663 / 6661 (55.0%) completed
3996 / 6661 (60.0%) completed
4329 / 6661 (65.0%) completed
4662 / 6661 (70.0%) completed
4995 / 6661 (75.0%) completed
5328 / 6661 (80.0%) completed
5661 / 6661 (85.0%) completed
5994 / 6661 (90.0%) completed
6327 / 6661 (95.0%) completed
6660 / 6661 (100.0%) completed


In [5]:
d = solver_data.score_dict
print(len(d), len(solver_data.answers))
for key in list(d.keys())[:10]:
    print(key, "".join(map(str, key[0])) + "=" + str(key[1]), len(d[key]))

6661 6661
98-4=94 9=8 6661
7-8+7=6 7=- 6661
75-3=72 7=5 6661
8-2*1=6 8=- 6661
51+5=56 5=1 6661
83-2=81 8=3 6661
23-20=3 2=3 6661
68/1=68 6=8 6661
9+5-9=5 9=+ 6661
94/47=2 9=4 6661


## Example Usage

In [11]:
score = nerdle.score_guess("1*1*6=6", "99/9=11")
print(score, 
      s.score_to_hint_string(score, NUM_SLOTS),
      s.score_to_hints(score, NUM_SLOTS) == [Hint.MISPLACED, Hint.INCORRECT, Hint.MISPLACED, Hint.INCORRECT, Hint.INCORRECT, Hint.MISPLACED, Hint.INCORRECT]
     )

2082 ?-?--?- True


In [20]:
# A good initial guess significantly reduces the number of answers. In this case, from
# 206 to 10.
guess_history, hint_history, answer_size_history =  nerdle.NerdleSolver(solver_data).solve("99/9=11", initial_guess= "1+6-7=0", debug=True)

guess 1+6-7=0 score ?----?- answers 128
guess 2*19=38 score --?++-- answers 1
guess 99/9=11 score +++++++ answers 1


## Benchmark
This is a fast in-memory dict implementation.

In [19]:
%time guess_history, hint_history, answer_size_history = nerdle.NerdleSolver(solver_data).solve("99/9=11", initial_guess="1+6-7=0")

CPU times: user 2.13 s, sys: 10.6 ms, total: 2.14 s
Wall time: 2.14 s


In [ ]:
answer = "99/9=11"
solver = nerdle.NerdleSolver(solver_data)
for start in list(solver_data.answers)[:5]:
    print(start)
    %time solver.solve(answer, initial_guess=start) 

## Initial Guess Optimization
Assuming something with a lot of different numbers and operators is best.

In [ ]:
# answer = "99/9=11"
# solutions = [nerdle.NerdleSolver(solver_data).solve(answer, initial_guess=start) 
#              for start in solver_data.answers]
# n = np.array([len(solution[0]) for solution in solutions])
# num_answers = len(solver_data.answers)
# compression_ratio = num_answers / np.array([solution[2][0] for solution in solutions])
# print(np.mean(compression_ratio), np.std(compression_ratio))

In [17]:
start = "1+6-7=0"
solutions = [nerdle.NerdleSolver(solver_data).solve(answer, initial_guess=start) 
             for answer in solver_data.answers]               
n = np.array([len(solution[0]) for solution in solutions])
num_answers = len(solver_data.answers)
compression_ratio = num_answers / np.array([solution[2][0] for solution in solutions])
print(np.mean(compression_ratio), np.std(compression_ratio))

fig, axs = plt.subplots(1, 2, figsize=(10, 4))

ax = axs[0]
ax.hist(n);
ax.set_title("#Guesses for start {}".format(start));

ax = axs[1]
ax.hist(compression_ratio);
ax.set_title("Compression Ratio Distribution, start {}".format(start));

KeyboardInterrupt: 